In [1]:
from misophonia_dataset._log import setup_print_logging

setup_print_logging()

import pandas as pd

from misophonia_dataset.source_data.esc50 import Esc50Dataset
from misophonia_dataset.source_data.fsd50k import Fsd50kDataset
from misophonia_dataset.source_data.foams import FoamsDataset

# Loading individual source data

In [2]:
esc50 = Esc50Dataset()
esc50.download_data()
esc50_meta = esc50.get_metadata()

esc50_meta.head()

,esc50_filename,esc50_fold,esc50_target,esc50_category,esc50_esc10,freesound_id,esc50_take,source_dataset,file_path,labels,label_type,licensing,validated_by,split
30,1-12653-A-15.wav,1,15,water_drops,False,12653,A,ESC50,/workspaces/misophonia-dataset/data/ESC50/ESC-...,[water_drops],trigger,({'license_url': 'http://creativecommons.org/l...,None,train
31,1-12654-A-15.wav,1,15,water_drops,False,12654,A,ESC50,/workspaces/misophonia-dataset/data/ESC50/ESC-...,[water_drops],trigger,({'license_url': 'http://creativecommons.org/l...,None,train
32,1-12654-B-15.wav,1,15,water_drops,False,12654,B,ESC50,/workspaces/misophonia-dataset/data/ESC50/ESC-...,[water_drops],trigger,({'license_url': 'http://creativecommons.org/l...,None,train
36,1-137-A-32.wav,1,32,keyboard_typing,False,137,A,ESC50,/workspaces/misophonia-dataset/data/ESC50/ESC-...,[typing],trigger,({'license_url': 'https://creativecommons.org/...,None,train
50,1-16746-A-15.wav,1,15,water_drops,False,16746,A,ESC50,/workspaces/misophonia-dataset/data/ESC50/ESC-...,[water_drops],trigger,({'license_url': 'https://creativecommons.org/...,None,val


In [3]:
fsd50k = Fsd50kDataset()
fsd50k.download_data()
fsd50k_meta = fsd50k.get_metadata()

assert fsd50k_meta.loc[fsd50k_meta["fsd50k_split"] == "eval", "split"].isin(["test"]).all(), (
    "All samples from FSD50K 'eval' must be in the 'test' split"
)

fsd50k_meta.head()

,fsd50k_fname,fsd50k_labels,fsd50k_mids,fsd50k_split,source_dataset,freesound_id,file_path,label_type,labels,licensing,validated_by,split
0,64760,[Electric_guitar],/m/02sgy,dev,FSD50K,64760,/workspaces/misophonia-dataset/data/FSD50K/dev...,background,[Electric_guitar],({'license_url': 'http://creativecommons.org/l...,None,train
1,16399,[Electric_guitar],/m/02sgy,dev,FSD50K,16399,/workspaces/misophonia-dataset/data/FSD50K/dev...,background,[Electric_guitar],({'license_url': 'http://creativecommons.org/l...,None,train
2,16401,[Electric_guitar],/m/02sgy,dev,FSD50K,16401,/workspaces/misophonia-dataset/data/FSD50K/dev...,background,[Electric_guitar],({'license_url': 'http://creativecommons.org/l...,None,train
3,16402,[Electric_guitar],/m/02sgy,dev,FSD50K,16402,/workspaces/misophonia-dataset/data/FSD50K/dev...,background,[Electric_guitar],({'license_url': 'http://creativecommons.org/l...,None,train
4,16404,[Electric_guitar],/m/02sgy,dev,FSD50K,16404,/workspaces/misophonia-dataset/data/FSD50K/dev...,background,[Electric_guitar],({'license_url': 'http://creativecommons.org/l...,None,train


In [4]:
foams = FoamsDataset()
foams.download_data()
foams_meta = foams.get_metadata()

assert foams_meta["split"].nunique() == 1 and foams_meta["split"].unique()[0] == "test", (
    "All FOAMS samples must be in the 'test' split"
)

foams_meta.head()

,freesound_id,labels,foams_duration,source_dataset,label_type,file_path,licensing,validated_by,split
0,184320,[basketball_dribbling],3.270003,FOAMS,trigger,/workspaces/misophonia-dataset/data/FOAMS/proc...,({'license_url': 'http://creativecommons.org/p...,"(FOAMS,)",test
1,580220,[basketball_dribbling],3.862957,FOAMS,trigger,/workspaces/misophonia-dataset/data/FOAMS/proc...,({'license_url': 'http://creativecommons.org/p...,"(FOAMS,)",test
2,262507,[basketball_dribbling],4.903607,FOAMS,trigger,/workspaces/misophonia-dataset/data/FOAMS/proc...,({'license_url': 'http://creativecommons.org/p...,"(FOAMS,)",test
3,453757,[basketball_dribbling],5.097981,FOAMS,trigger,/workspaces/misophonia-dataset/data/FOAMS/proc...,({'license_url': 'http://creativecommons.org/p...,"(FOAMS,)",test
4,530362,[basketball_dribbling],5.344224,FOAMS,trigger,/workspaces/misophonia-dataset/data/FOAMS/proc...,({'license_url': 'http://creativecommons.org/p...,"(FOAMS,)",test


# Combining all source data

In [5]:
combined_meta = pd.concat([esc50_meta, fsd50k_meta, foams_meta], ignore_index=True)
combined_meta = combined_meta[
    # These colums are there, see the interface.SourceMetaData class
    ["source_dataset", "split", "freesound_id", "label_type", "labels", "validated_by", "file_path", "licensing"] 
]

## Examine different subsets

In [6]:
from IPython.display import HTML, display

def print_value_counts_pcts(title: str, col: pd.Series) -> None:
    normalized = col.value_counts(normalize=True, dropna=False)
    counts = col.value_counts(dropna=False)
    # if all the same, just print that
    if len(counts) == 1:
        print(f"{title} All samples have the same value: {counts.index[0]}")
        return
    
    dist = pd.DataFrame({"count": counts, "percentage": normalized * 100}, index=counts.index)
    print(title)
    display(dist)

def print_subset_info(subset_descr: str, subset: pd.DataFrame):
    display(HTML(f"<h3>=== Subset: {subset_descr} ===</h3>"))
    print(f"* Total samples: {len(subset)}")
    print_value_counts_pcts("\n* Distribution by source dataset:", subset["source_dataset"])
    print_value_counts_pcts("\n* Distribution by split:", subset["split"])
    print_value_counts_pcts("\n* Distribution by label type:", subset["label_type"])
    print_value_counts_pcts("\n* Distribution by validated by:", subset["validated_by"])
    print("\n* Sample entries:")
    display(subset.sample(min(3, len(subset))))

### Examine all source data

In [7]:
print_subset_info("All source data", combined_meta)

* Total samples: 31951

* Distribution by source dataset:


,count,percentage
source_dataset,,
FSD50K,31701,99.217552
ESC50,200,0.625958
FOAMS,50,0.156490



* Distribution by split:


,count,percentage
split,,
train,23993,75.093111
test,4697,14.700635
val,3261,10.206253



* Distribution by label type:


,count,percentage
label_type,,
background,28640,89.637257
trigger,2417,7.564708
control,894,2.798034



* Distribution by validated by:


,count,percentage
validated_by,,
None,31889,99.805953
"(FOAMS,)",62,0.194047



* Sample entries:


,source_dataset,split,freesound_id,label_type,labels,validated_by,file_path,licensing
21298,FSD50K,val,208689,background,[Zipper_(clothing)],None,/workspaces/misophonia-dataset/data/FSD50K/dev...,({'license_url': 'http://creativecommons.org/p...
12983,FSD50K,train,138856,background,[Hi-hat],None,/workspaces/misophonia-dataset/data/FSD50K/dev...,({'license_url': 'http://creativecommons.org/l...
13224,FSD50K,train,323223,background,[Drum_kit],None,/workspaces/misophonia-dataset/data/FSD50K/dev...,({'license_url': 'http://creativecommons.org/l...


### Examine by label type

In [8]:
# For each label type
for label_type in combined_meta["label_type"].unique():
    subset = combined_meta[combined_meta["label_type"] == label_type]
    print_subset_info(f"label_type = {label_type}", subset)

* Total samples: 2417

* Distribution by source dataset:


,count,percentage
source_dataset,,
FSD50K,2167,89.656599
ESC50,200,8.274721
FOAMS,50,2.068680



* Distribution by split:


,count,percentage
split,,
train,1646,68.100952
test,548,22.672735
val,223,9.226314



* Distribution by label type: All samples have the same value: trigger

* Distribution by validated by:


...

,source_dataset,split,freesound_id,label_type,labels,validated_by,file_path,licensing
5779,FSD50K,train,355300,background,[Oboe],None,/workspaces/misophonia-dataset/data/FSD50K/dev...,({'license_url': 'http://creativecommons.org/l...
5822,FSD50K,train,119265,background,[Alto_saxophone],None,/workspaces/misophonia-dataset/data/FSD50K/dev...,({'license_url': 'http://creativecommons.org/l...
2477,FSD50K,train,343751,background,[Speech],None,/workspaces/misophonia-dataset/data/FSD50K/dev...,({'license_url': 'http://creativecommons.org/l...


* Total samples: 894

* Distribution by source dataset: All samples have the same value: FSD50K

* Distribution by split:


,count,percentage
split,,
train,675,75.503356
test,128,14.317673
val,91,10.178971



* Distribution by label type: All samples have the same value: control

* Distribution by validated by: All samples have the same value: None

* Sample entries:


,source_dataset,split,freesound_id,label_type,labels,validated_by,file_path,licensing
14410,FSD50K,train,148481,control,[piano],None,/workspaces/misophonia-dataset/data/FSD50K/dev...,({'license_url': 'http://creativecommons.org/l...
29627,FSD50K,test,121078,control,[piano],None,/workspaces/misophonia-dataset/data/FSD50K/eva...,({'license_url': 'http://creativecommons.org/l...
27207,FSD50K,train,272075,control,[hair_dryer],None,/workspaces/misophonia-dataset/data/FSD50K/dev...,({'license_url': 'http://creativecommons.org/l...


### Examine by trigger category

In [9]:
# For each trigger sound label
for label in combined_meta[combined_meta["label_type"] == "trigger"]["labels"].explode().unique():
    subset = combined_meta[
        (combined_meta["label_type"] == "trigger") & (combined_meta["labels"].apply(lambda labels: label in labels))
    ]
    print_subset_info(f"Trigger = {label}", subset)

* Total samples: 328

* Distribution by source dataset:


,count,percentage
source_dataset,,
FSD50K,283,86.280488
ESC50,40,12.195122
FOAMS,5,1.524390



* Distribution by split:


,count,percentage
split,,
train,230,70.121951
test,63,19.207317
val,35,10.670732



* Distribution by label type: All samples have the same value: trigger

* Distribution by validated by:


...

,source_dataset,split,freesound_id,label_type,labels,validated_by,file_path,licensing
22435,FSD50K,train,331160,trigger,[knife_cutting],None,/workspaces/misophonia-dataset/data/FSD50K/dev...,({'license_url': 'http://creativecommons.org/l...
30915,FSD50K,test,81119,trigger,[knife_cutting],None,/workspaces/misophonia-dataset/data/FSD50K/eva...,({'license_url': 'http://creativecommons.org/p...
22413,FSD50K,train,240473,trigger,[knife_cutting],None,/workspaces/misophonia-dataset/data/FSD50K/dev...,({'license_url': 'http://creativecommons.org/p...


* Total samples: 5

* Distribution by source dataset: All samples have the same value: FOAMS

* Distribution by split: All samples have the same value: test

* Distribution by label type: All samples have the same value: trigger

* Distribution by validated by: All samples have the same value: ('FOAMS',)

* Sample entries:


,source_dataset,split,freesound_id,label_type,labels,validated_by,file_path,licensing
31901,FOAMS,test,184320,trigger,[basketball_dribbling],"(FOAMS,)",/workspaces/misophonia-dataset/data/FOAMS/proc...,({'license_url': 'http://creativecommons.org/p...
31902,FOAMS,test,580220,trigger,[basketball_dribbling],"(FOAMS,)",/workspaces/misophonia-dataset/data/FOAMS/proc...,({'license_url': 'http://creativecommons.org/p...
31904,FOAMS,test,453757,trigger,[basketball_dribbling],"(FOAMS,)",/workspaces/misophonia-dataset/data/FOAMS/proc...,({'license_url': 'http://creativecommons.org/p...


* Total samples: 5

* Distribution by source dataset: All samples have the same value: FOAMS

* Distribution by split: All samples have the same value: test

* Distribution by label type: All samples have the same value: trigger

* Distribution by validated by: All samples have the same value: ('FOAMS',)

* Sample entries:


,source_dataset,split,freesound_id,label_type,labels,validated_by,file_path,licensing
31918,FOAMS,test,484956,trigger,[flipping_newspaper],"(FOAMS,)",/workspaces/misophonia-dataset/data/FOAMS/proc...,({'license_url': 'http://creativecommons.org/p...
31916,FOAMS,test,636467,trigger,[flipping_newspaper],"(FOAMS,)",/workspaces/misophonia-dataset/data/FOAMS/proc...,({'license_url': 'http://creativecommons.org/p...
31917,FOAMS,test,473981,trigger,[flipping_newspaper],"(FOAMS,)",/workspaces/misophonia-dataset/data/FOAMS/proc...,({'license_url': 'http://creativecommons.org/p...


### Examine by source dataset

In [10]:
# for each source dataset
for source_dataset in combined_meta["source_dataset"].unique():
    subset = combined_meta[combined_meta["source_dataset"] == source_dataset]
    print_subset_info(f"Source dataset = {source_dataset}", subset)

* Total samples: 200

* Distribution by source dataset: All samples have the same value: ESC50

* Distribution by split:


,count,percentage
split,,
train,131,65.5
test,53,26.5
val,16,8.0



* Distribution by label type: All samples have the same value: trigger

* Distribution by validated by:


,count,percentage
validated_by,,
None,194,97.0
"(FOAMS,)",6,3.0



* Sample entries:


...

,count,percentage
validated_by,,
None,31695,99.981073
"(FOAMS,)",6,0.018927



* Sample entries:


,source_dataset,split,freesound_id,label_type,labels,validated_by,file_path,licensing
1850,FSD50K,train,236560,background,[Male_speech_and_man_speaking],None,/workspaces/misophonia-dataset/data/FSD50K/dev...,({'license_url': 'http://creativecommons.org/l...
11415,FSD50K,train,35592,background,[Snare_drum],None,/workspaces/misophonia-dataset/data/FSD50K/dev...,({'license_url': 'http://creativecommons.org/l...
6193,FSD50K,train,248453,background,[Clarinet],None,/workspaces/misophonia-dataset/data/FSD50K/dev...,({'license_url': 'http://creativecommons.org/l...


* Total samples: 50

* Distribution by source dataset: All samples have the same value: FOAMS

* Distribution by split: All samples have the same value: test

* Distribution by label type: All samples have the same value: trigger

* Distribution by validated by: All samples have the same value: ('FOAMS',)

* Sample entries:


,source_dataset,split,freesound_id,label_type,labels,validated_by,file_path,licensing
31915,FOAMS,test,592495,trigger,[clearing_throat],"(FOAMS,)",/workspaces/misophonia-dataset/data/FOAMS/proc...,({'license_url': 'http://creativecommons.org/p...
31904,FOAMS,test,453757,trigger,[basketball_dribbling],"(FOAMS,)",/workspaces/misophonia-dataset/data/FOAMS/proc...,({'license_url': 'http://creativecommons.org/p...
31908,FOAMS,test,125375,trigger,[chewing_gum],"(FOAMS,)",/workspaces/misophonia-dataset/data/FOAMS/proc...,({'license_url': 'http://creativecommons.org/p...


### Examine single vs. multi-label data

In [11]:
def normalize_label_tuple(s: pd.Series) -> pd.Series:
    return s.apply(lambda labels: tuple(sorted(labels)))

label_len = combined_meta["labels"].apply(len)
for label_count in sorted(label_len.unique()):
    subset = combined_meta[label_len == label_count]
    print_subset_info(f"Number of labels = {label_count}", subset)
    print_value_counts_pcts("\n* Label pairs distribution:", normalize_label_tuple(subset["labels"]))

    print("\n ---- ")

* Total samples: 28752

* Distribution by source dataset:


,count,percentage
source_dataset,,
FSD50K,28502,99.130495
ESC50,200,0.695604
FOAMS,50,0.173901



* Distribution by split:


,count,percentage
split,,
train,22145,77.020729
test,3610,12.555648
val,2997,10.423623



* Distribution by label type:


...

* Total samples: 2

* Distribution by source dataset: All samples have the same value: FSD50K

* Distribution by split: All samples have the same value: test

* Distribution by label type: All samples have the same value: background

* Distribution by validated by: All samples have the same value: None

* Sample entries:


,source_dataset,split,freesound_id,label_type,labels,validated_by,file_path,licensing
28494,FSD50K,test,431330,background,"[Wind_instrument_and_woodwind_instrument, Bras...",None,/workspaces/misophonia-dataset/data/FSD50K/eva...,({'license_url': 'http://creativecommons.org/l...
28254,FSD50K,test,368610,background,"[Trombone, Trumpet, Bass_drum, Cowbell, Synthe...",None,/workspaces/misophonia-dataset/data/FSD50K/eva...,({'license_url': 'http://creativecommons.org/l...



* Label pairs distribution:


,count,percentage
labels,,
"(Bass_drum, Cowbell, Hi-hat, Snare_drum, Synthesizer, Trombone, Trumpet)",1,50.0
"(Brass_instrument, Cello, Double_bass, Drum, Orchestra, Pizzicato, Wind_instrument_and_woodwind_instrument)",1,50.0



 ---- 


### Examine single vs. multi-label trigger data

In [12]:
trigger_subset = combined_meta[combined_meta["label_type"] == "trigger"]
label_len = trigger_subset["labels"].apply(len)
for label_count in sorted(label_len.unique()):
    subset = trigger_subset[label_len == label_count]
    print_subset_info(f"Number of labels = {label_count}", subset)
    print_value_counts_pcts("\n* Label pairs distribution:", normalize_label_tuple(subset["labels"]))

    print("\n ---- ")

* Total samples: 2389

* Distribution by source dataset:


,count,percentage
source_dataset,,
FSD50K,2139,89.535370
ESC50,200,8.371704
FOAMS,50,2.092926



* Distribution by split:


,count,percentage
split,,
train,1641,68.689828
test,525,21.975722
val,223,9.334450



* Distribution by label type: All samples have the same value: trigger

* Distribution by validated by:


...


* Distribution by label type: All samples have the same value: trigger

* Distribution by validated by: All samples have the same value: None

* Sample entries:


,source_dataset,split,freesound_id,label_type,labels,validated_by,file_path,licensing
29962,FSD50K,test,218411,trigger,"[clearing_throat, human_breathing]",None,/workspaces/misophonia-dataset/data/FSD50K/eva...,({'license_url': 'http://creativecommons.org/p...
29958,FSD50K,test,169217,trigger,"[clearing_throat, human_breathing]",None,/workspaces/misophonia-dataset/data/FSD50K/eva...,({'license_url': 'http://creativecommons.org/l...
17921,FSD50K,train,337506,trigger,"[water_drops, swallowing]",None,/workspaces/misophonia-dataset/data/FSD50K/dev...,({'license_url': 'http://creativecommons.org/l...



* Label pairs distribution:


,count,percentage
labels,,
"(clearing_throat, human_breathing)",18,64.285714
"(chewing_gum, human_breathing)",4,14.285714
"(chewing_gum, clearing_throat)",2,7.142857
"(swallowing, water_drops)",2,7.142857
"(chewing_gum, plastic_crumpling)",2,7.142857



 ---- 
